In [1]:
from core.libs import json

# 1. Ruta al schema JSON
schema_path = r"C:\Users\HeyCe\World Tree Technologies Inc\Operations - Documentos\WorldTreeSystem\AuxiliaryScripts\inventory_metrics_schema.json"
with open(schema_path, "r", encoding="utf-8") as f:
    schema = json.load(f)

In [2]:
newworldorder = ["rel_path","contract_code","inventory_year","inventory_date","survival","tht_mean","tht_std","mht_mean","mht_std","mht_pct_of_target","dbh_mean","dbh_std","dbh_pct_of_target","doyle_bf_mean","doyle_bf_std","doyle_bf_total","projected_dbh","projected_doyle_bf"]


In [5]:
# 3. Haz un dict rápido {col_name: propiedades}
schema_map = {col["column_name"]: col for col in schema}

# 4. Utilidad para armar la línea SQL de cada columna
def make_sql_line(col):
    name = col["column_name"]
    dtype = col["data_type"].upper()
    # Numeric precision/scale
    if dtype == "NUMERIC":
        if col["numeric_precision"] and col["numeric_scale"] is not None:
            dtype += f'({col["numeric_precision"]},{col["numeric_scale"]})'
    elif dtype == "INTEGER" and col["numeric_precision"]:
        # Opcional: puedes ignorar (PostgreSQL ya lo interpreta)
        pass
    # Nullabilidad
    nullable = "NOT NULL" if col["is_nullable"] == "NO" else ""
    # Default
    default = f'DEFAULT {col["column_default"]}' if col["column_default"] else ""
    return f"    {name} {dtype} {nullable} {default}".strip()

for colname in newworldorder:
    print(make_sql_line(schema_map[colname]))

rel_path TEXT
contract_code TEXT NOT NULL
inventory_year INTEGER
inventory_date TEXT NOT NULL
survival NUMERIC(3,2)
tht_mean NUMERIC
tht_std NUMERIC
mht_mean NUMERIC
mht_std NUMERIC
mht_pct_of_target NUMERIC
dbh_mean NUMERIC
dbh_std NUMERIC
dbh_pct_of_target NUMERIC
doyle_bf_mean NUMERIC
doyle_bf_std NUMERIC
doyle_bf_total NUMERIC
projected_dbh NUMERIC
projected_doyle_bf NUMERIC


In [9]:
from sqlalchemy import text
from core.db import get_engine
engine = get_engine()

# 5. Construir el CREATE TABLE ordenado
lines = [make_sql_line(schema_map[col]) for col in newworldorder]

create_stmt = (
    "CREATE TABLE masterdatabase.inventory_metrics_new (\n" +
    ",\n".join(lines) +
    "\n);"
)

print(create_stmt)

with engine.connect() as conn:
    conn.execute(text(create_stmt))
    conn.commit()
print("✅ Tabla creada en la base.")

💻 Conectado a la base de datos helloworldtree
CREATE TABLE masterdatabase.inventory_metrics_new (
rel_path TEXT,
contract_code TEXT NOT NULL,
inventory_year INTEGER,
inventory_date TEXT NOT NULL,
survival NUMERIC(3,2),
tht_mean NUMERIC,
tht_std NUMERIC,
mht_mean NUMERIC,
mht_std NUMERIC,
mht_pct_of_target NUMERIC,
dbh_mean NUMERIC,
dbh_std NUMERIC,
dbh_pct_of_target NUMERIC,
doyle_bf_mean NUMERIC,
doyle_bf_std NUMERIC,
doyle_bf_total NUMERIC,
projected_dbh NUMERIC,
projected_doyle_bf NUMERIC
);
✅ Tabla creada en la base.


In [14]:
from core.db import get_engine
from sqlalchemy import text

engine = get_engine()


# 2. Copia los datos al nuevo orden
insert_sql = """
INSERT INTO masterdatabase.inventory_metrics_new (
    rel_path,
    contract_code,
    inventory_year,
    inventory_date,
    survival,
    tht_mean,
    tht_std,
    mht_mean,
    mht_std,
    mht_pct_of_target,
    dbh_mean,
    dbh_std,
    dbh_pct_of_target,
    doyle_bf_mean,
    doyle_bf_std,
    doyle_bf_total,
    projected_dbh,
    projected_doyle_bf
)
SELECT
    rel_path,
    contract_code,
    inventory_year,
    inventory_date,
    survival,
    tht_mean,
    tht_std,
    mht_mean,
    mht_std,
    mht_pct_of_target,
    dbh_mean,
    dbh_std,
    dbh_pct_of_target,
    doyle_bf_mean,
    doyle_bf_std,
    doyle_bf_total,
    projected_dbh,
    projected_doyle_bf
FROM masterdatabase.inventory_metrics;
"""

# 3. Haz el swap (si quieres reemplazar la original)
swap_sql = """
DROP TABLE masterdatabase.inventory_metrics;
ALTER TABLE masterdatabase.inventory_metrics_new RENAME TO inventory_metrics;
"""

with engine.begin() as conn:  # begin() hace commit automático
    print("🚧 Copiando datos...")
    conn.execute(text(insert_sql))
    # Si quieres hacer el swap, descomenta:
    print("🚧 Reemplazando tabla original (swap)...")
    conn.execute(text(swap_sql))

print("✅ ¡Proceso terminado, padrino!")


💻 Conectado a la base de datos helloworldtree
🚧 Copiando datos...
🚧 Reemplazando tabla original (swap)...
✅ ¡Proceso terminado, padrino!
